In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from pathlib import Path
from sklearn import metrics
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import torchvision

from datetime import datetime
from collections import OrderedDict

In [2]:
import pickle

In [3]:
PATH = Path("/data2/yinterian/multi-task-romain")

In [4]:
gap = "15min"
gap

'15min'

In [5]:
filename = "data_train_{gap}.pickle".format(gap=gap)
with open(PATH/filename, 'rb') as f:
    train = pickle.load(f)

In [6]:
filename = "data_valid_{gap}.pickle".format(gap=gap)
with open(PATH/filename, 'rb') as f:
    valid = pickle.load(f)

In [7]:
train.shape, valid.shape

((42830, 14), (5069, 14))

In [8]:
subject_id_list = np.sort(np.unique(train.subject_id.values))
id2index = {v: k+1 for k,v in enumerate(subject_id_list)}
num_subjects = len(subject_id_list)

In [9]:
num_subjects

2170

## Dataset

In [10]:
def get_mean_std_series(train):
    ss = np.concatenate(train.series.values)
    ss = ss.reshape(-1,5)
    return ss.mean(axis=0), ss.std(axis=0)

In [11]:
def get_mean_std_static(train):
    res = {}
    for name in ["age", "sapsii", "sofa"]:
        values = train[name].values
        res[name] = (values.mean(), values.std())
    res["series"] = get_mean_std_series(train)
    return res

In [12]:
norm_dict = get_mean_std_static(train)
norm_dict 

{'age': (64, 15.087455295966063),
 'sapsii': (33, 14.265492481117855),
 'sofa': (4, 3.7831641172054082),
 'series': (array([ 83.19453341,  93.64397046, 121.07613603,  58.73969887,
          78.6694367 ]),
  array([16.08727268, 17.53684697, 21.3399693 , 12.26982071, 14.36323955]))}

In [13]:
care2id = {v:k for k,v in enumerate(np.unique(train.care_unit.values))}
care2id 

{1: 0, 2: 1, 3: 2, 5: 3, 6: 4}

In [69]:
class MultiTask(Dataset):
    def __init__(self, df, norm_dict, id2index, care2id, k=20, train=True):
        """
        Args:
            df: dataframe with data
            norm_dict: mean and std of all variables to normalize
            
        """
        self.norm_dict = norm_dict
        self.df = df
        self.df["care_unit"] = self.df["care_unit"].apply(lambda x: care2id[x])
        self.names = ["age", "sapsii", "sofa"] ## needs normalization
        self.names_binary = ["gender", "amine", "sedation", "ventilation"]
        self.id2index = id2index
        self.train = train
        self.df_sample = self.pick_a_sample(k)
            
    def pick_a_sample(self, k=20):
        """ Picks sample with the same number of observations per patient"""
        #if not self.train: # fix seed for validation and test
        #    np.random.seed(3)
        sample = self.df.groupby("subject_id", group_keys=False).apply(lambda x: x.sample(min(len(x), k)))
        sample = sample.copy()
        if self.train:
            self.subject_index = [self.id2index[subject_id] for subject_id in sample.subject_id.values]
            self.random = np.random.choice(2, sample.shape[0], p=[0.1, 0.9])
            self.subject_index = self.subject_index*self.random
        return sample

    def __getitem__(self, index):
        row = self.df_sample.iloc[index,:]
        x_series = (row.series - self.norm_dict["series"][0])/self.norm_dict["series"][1]
        x_cont = [(row[name]-self.norm_dict[name][0])/self.norm_dict[name][1] for name in self.names]
        x_binary = [row[name] for name in self.names_binary]
        subject_index = 0
        if self.train:
            subject_index = self.subject_index[index]
        x_cat = np.array([row["care_unit"], subject_index])
        x_cont = np.array(x_cont + x_binary)
        return x_series, x_cont, x_cat, row["prediction_mean_HR"], row["prediction_mean_MAP"]

    def __len__(self):
        return self.df_sample.shape[0]

In [16]:
train_ds = MultiTask(train, norm_dict, id2index, care2id)
valid_ds = MultiTask(valid, norm_dict, id2index, care2id, train=False)

## Model

In [17]:
def save_model(m, p): torch.save(m.state_dict(), p)
    
def load_model(m, p): m.load_state_dict(torch.load(p))

In [18]:
def val_metrics(model, valid_dl, which_y="y1"):
    model.eval()
    total = 0
    sum_loss = 0
    y_hat = []
    ys = []
    for x_series, x_cont, x_cat, y1, y2 in valid_dl:
        batch = y1.shape[0]
        x_series = x_series.float().cuda()
        x_cont = x_cont.float().cuda()
        x_cat = x_cat.long().cuda()
        y1 = y1.float().cuda()
        y2 = y2.float().cuda()
        out = model(x_series, x_cont, x_cat)
        if which_y=="y1":
            mse_loss = F.mse_loss(out, y1.unsqueeze(-1))
            ys.append(y1.view(-1).cpu().numpy())
        else:
            mse_loss = F.mse_loss(out, y2.unsqueeze(-1))
            ys.append(y2.view(-1).cpu().numpy())
        sum_loss += batch*(mse_loss.item())
        total += batch
        y_hat.append(out.view(-1).detach().cpu().numpy())
    
    y_hat = np.concatenate(y_hat)
    ys = np.concatenate(ys)
    r2 = metrics.r2_score(ys, y_hat)
    
    return sum_loss/total, r2

In [19]:
def train_epochs(model, train_ds, optimizer, lr=1e-3, epochs = 30, which_y="y1"):
    prev_val_r2 = 0
    for i in range(epochs):
        sum_loss = 0
        total = 0
        train_ds.pick_a_sample()
        train_dl = DataLoader(train_ds, batch_size=5000, shuffle=True)
        for x_series, x_cont, x_cat, y1, y2 in train_dl:
            model.train()
            x_series = x_series.float().cuda()
            x_cont = x_cont.float().cuda()
            x_cat = x_cat.long().cuda()
            y1 = y1.float().cuda()
            y2 = y2.float().cuda()
            out = model(x_series, x_cont, x_cat)
            if which_y=="y1":
                loss = F.mse_loss(out, y1.unsqueeze(-1))
            else:
                loss = F.mse_loss(out, y2.unsqueeze(-1))
            sum_loss += len(y1) * loss.item()
            
            total += len(y1)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if i % 1 == 0:
            val_loss, val_r2= val_metrics(model, valid_dl, which_y=which_y)
            print("\tTrain loss: {:.3f} valid loss: {:.3f} valid r2 {:.3f}".format(
                sum_loss/total, val_loss, val_r2))
        if val_r2 > prev_val_r2:
            prev_val_r2 = val_r2
            if val_r2 > 0.7:
                filename = "single_model_" + which_y
                path = "{0}/models/{1}_r2_{2:.0f}.pth".format(PATH, filename, 100*val_r2) 
                save_model(model, path)
                print(path)

In [26]:
batch_size = 5000
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)

## Model 3

In [21]:
class EventModel3(nn.Module):
    def __init__(self, hidden_size=100, num2=50, num_units=5):
        super(EventModel3, self).__init__()
        self.embedding1 = nn.Embedding(num_units, 1)
        self.embedding2 = nn.Embedding(num_subjects+1, 5)
        self.gru = nn.GRU(5, hidden_size, batch_first=True)
        self.num1 = hidden_size + 1 + 5 + 7
        self.num2 = num2
        self.linear1 = nn.Linear(self.num1, self.num2)
        self.linear2 = nn.Linear(self.num2, self.num2)
        self.out = nn.Linear(self.num2, 1)
        self.bn1 = nn.BatchNorm1d(self.num2)
        self.bn2 = nn.BatchNorm1d(self.num2)
        
    def forward(self, x_series, x_cont, x_cat):
        _, ht = self.gru(x_series)
        x_cat_1 = self.embedding1(x_cat[:,0])
        x_cat_2 = self.embedding2(x_cat[:,1])
        x = torch.cat((ht[-1], x_cat_1, x_cat_2, x_cont), 1)
        x = self.bn1(F.relu(self.linear1(x)))
        x = self.bn2(F.relu(self.linear2(x)))
        return self.out(x)

In [22]:
model = EventModel3().cuda()

In [23]:
x_series, x_cont, x_cat, y_HR, y_MAP = next(iter(train_dl))

In [24]:
x_series = x_series.float().cuda()
x_cont = x_cont.float().cuda()
x_cat = x_cat.long().cuda()

In [25]:
y = model(x_series, x_cont, x_cat)

In [27]:
# model for mean_HR
model = EventModel3().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.03, weight_decay=1e-5)
train_epochs(model, train_ds, optimizer, epochs=15)

	Train loss: 7042.043 valid loss: 6933.797 valid r2 -26.036
	Train loss: 6563.981 valid loss: 4335.460 valid r2 -15.905
	Train loss: 5863.749 valid loss: 4535.086 valid r2 -16.683
	Train loss: 4907.551 valid loss: 4177.601 valid r2 -15.289
	Train loss: 3663.166 valid loss: 2694.306 valid r2 -9.506
	Train loss: 2174.319 valid loss: 1101.661 valid r2 -3.296
	Train loss: 861.754 valid loss: 204.086 valid r2 0.204
	Train loss: 146.929 valid loss: 28.701 valid r2 0.888
/data2/yinterian/multi-task-romain/models/single_model_y1_r2_89.pth
	Train loss: 56.943 valid loss: 253.329 valid r2 0.012
	Train loss: 199.002 valid loss: 274.183 valid r2 -0.069
	Train loss: 166.185 valid loss: 101.982 valid r2 0.602
	Train loss: 51.219 valid loss: 17.630 valid r2 0.931
/data2/yinterian/multi-task-romain/models/single_model_y1_r2_93.pth
	Train loss: 13.338 valid loss: 16.924 valid r2 0.934
/data2/yinterian/multi-task-romain/models/single_model_y1_r2_93.pth
	Train loss: 27.988 valid loss: 32.647 valid r2 0.8

In [28]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=1e-5)
train_epochs(model, train_ds, optimizer, epochs=15)

	Train loss: 21.012 valid loss: 45.213 valid r2 0.824
/data2/yinterian/multi-task-romain/models/single_model_y1_r2_82.pth
	Train loss: 14.094 valid loss: 38.205 valid r2 0.851
/data2/yinterian/multi-task-romain/models/single_model_y1_r2_85.pth
	Train loss: 12.965 valid loss: 19.784 valid r2 0.923
/data2/yinterian/multi-task-romain/models/single_model_y1_r2_92.pth
	Train loss: 11.700 valid loss: 15.851 valid r2 0.938
/data2/yinterian/multi-task-romain/models/single_model_y1_r2_94.pth
	Train loss: 10.837 valid loss: 14.753 valid r2 0.942
/data2/yinterian/multi-task-romain/models/single_model_y1_r2_94.pth
	Train loss: 10.553 valid loss: 12.614 valid r2 0.951
/data2/yinterian/multi-task-romain/models/single_model_y1_r2_95.pth
	Train loss: 10.739 valid loss: 10.908 valid r2 0.957
/data2/yinterian/multi-task-romain/models/single_model_y1_r2_96.pth
	Train loss: 10.512 valid loss: 12.008 valid r2 0.953
	Train loss: 10.671 valid loss: 15.540 valid r2 0.939
	Train loss: 10.669 valid loss: 12.326

In [29]:
# mean_MAP
model = EventModel3().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.03, weight_decay=1e-5)
train_epochs(model, train_ds, optimizer, epochs=15, which_y="y2")

	Train loss: 6202.056 valid loss: 7171.359 valid r2 -38.460
	Train loss: 5802.007 valid loss: 6970.132 valid r2 -37.352
	Train loss: 5181.522 valid loss: 6550.658 valid r2 -35.044
	Train loss: 4304.595 valid loss: 4648.040 valid r2 -24.575
	Train loss: 3150.987 valid loss: 2800.306 valid r2 -14.408
	Train loss: 1827.713 valid loss: 937.386 valid r2 -4.158
	Train loss: 658.341 valid loss: 173.789 valid r2 0.044
	Train loss: 93.757 valid loss: 33.124 valid r2 0.818
/data2/yinterian/multi-task-romain/models/single_model_y2_r2_82.pth
	Train loss: 88.739 valid loss: 109.755 valid r2 0.396
	Train loss: 205.611 valid loss: 132.688 valid r2 0.270
	Train loss: 136.892 valid loss: 44.479 valid r2 0.755
	Train loss: 38.684 valid loss: 18.951 valid r2 0.896
/data2/yinterian/multi-task-romain/models/single_model_y2_r2_90.pth
	Train loss: 23.894 valid loss: 35.686 valid r2 0.804
	Train loss: 38.813 valid loss: 36.549 valid r2 0.799
	Train loss: 35.036 valid loss: 25.053 valid r2 0.862


In [30]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=1e-5)
train_epochs(model, train_ds, optimizer, epochs=15, which_y="y2")

	Train loss: 24.718 valid loss: 32.093 valid r2 0.823
/data2/yinterian/multi-task-romain/models/single_model_y2_r2_82.pth
	Train loss: 20.594 valid loss: 27.351 valid r2 0.850
/data2/yinterian/multi-task-romain/models/single_model_y2_r2_85.pth
	Train loss: 19.094 valid loss: 30.055 valid r2 0.835
	Train loss: 18.521 valid loss: 18.130 valid r2 0.900
/data2/yinterian/multi-task-romain/models/single_model_y2_r2_90.pth
	Train loss: 18.064 valid loss: 18.547 valid r2 0.898
	Train loss: 17.512 valid loss: 16.253 valid r2 0.911
/data2/yinterian/multi-task-romain/models/single_model_y2_r2_91.pth
	Train loss: 17.135 valid loss: 16.223 valid r2 0.911
/data2/yinterian/multi-task-romain/models/single_model_y2_r2_91.pth
	Train loss: 16.969 valid loss: 16.010 valid r2 0.912
/data2/yinterian/multi-task-romain/models/single_model_y2_r2_91.pth
	Train loss: 16.671 valid loss: 16.593 valid r2 0.909
	Train loss: 16.403 valid loss: 16.338 valid r2 0.910
	Train loss: 16.324 valid loss: 17.176 valid r2 0.90

## Test 

In [31]:
path = PATH/"models/single_model_y1_r2_96.pth"
model = EventModel3().cuda()
load_model(model, path)

filename = "data_test_{gap}.pickle".format(gap=gap)
with open(PATH/filename, 'rb') as f:
    test = pickle.load(f)
print(filename)

In [33]:
test_ds = MultiTask(test, norm_dict, id2index, care2id, k=25, train=False)
test.shape, len(test_ds)

((5933, 14), 3794)

In [34]:
test_dl = DataLoader(test_ds, batch_size=4561)

In [35]:
val_metrics(model, valid_dl, which_y="y1")

(10.908259391784668, 0.9574664495909214)

In [36]:
path = PATH/"models/single_model_y2_r2_91.pth"
load_model(model, path)

In [37]:
val_metrics(model, valid_dl, which_y="y2")

(16.00960922241211, 0.9119090799797106)

In [38]:
(0.957 + 0.91)/2

0.9335

## External data

In [61]:
def get_r2_with_ci(model, df, which_y, n=1000):
    losses = []
    r2s = []
    for i in range(n):
        test_ext_ds = MultiTask(df.copy(), norm_dict, id2index, care2id, k=13, train=False)
        test_ext_dl = DataLoader(test_ext_ds, batch_size=len(test_ext_ds))
        loss, r2 = val_metrics(model, test_ext_dl, which_y=which_y)
        losses.append(loss)
        r2s.append(r2)
    return losses, r2s

In [70]:
filename = "data_validation_15min.pickle"
with open(PATH/filename, 'rb') as f:
    test_ext = pickle.load(f)

In [71]:
path = PATH/"models/single_model_y1_r2_96.pth"
model = EventModel3().cuda()
load_model(model, path)

In [72]:
losses, r2s = get_r2_with_ci(model, test_ext, which_y="y1")

In [73]:
path = PATH/"models/single_model_y2_r2_91.pth"
load_model(model, path)

In [74]:
losses_y2, r2s_y2 = get_r2_with_ci(model, test_ext, which_y="y2")

In [75]:
np.quantile(r2s, [0.025, 0.5, 0.975])

array([0.82874327, 0.85672872, 0.88146229])

In [76]:
np.quantile(r2s_y2, [0.025, 0.5, 0.975])

array([0.73040045, 0.76939074, 0.80461116])